In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

model = gp.Model(name="quadratic_program")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-28


In [2]:
# data
corr_data = np.matrix([ [1, 0.6, 0.2], 
                        [0.6, 1, 0.5],
                        [0.2, 0.5, 1]])

std_data = np.array([[0.12],
                     [0.20], 
                     [0.05]])

std_data_T = np.array([[0.12],
                     [0.20], 
                     [0.05]]).transpose()

In [3]:
std_data_T.shape

(1, 3)

In [4]:
# calculate covariance matrix
v1 = np.matmul(std_data, std_data_T)
v = np.multiply(corr_data, v1) 
print(v)

[[0.0144 0.0144 0.0012]
 [0.0144 0.04   0.005 ]
 [0.0012 0.005  0.0025]]


In [5]:
# Percentage of the portfolio invested in asset i
x_1 = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x1")
x_2 = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x2")
x_3 = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x3")

In [6]:
# x = np.array([[x_1],
#               [x_2],
#               [x_3]])
# x_T = x.transpose()

In [18]:
x = np.array([x_1, x_2, x_3]).reshape(3,1)
x_T = x.transpose()

In [8]:
# obj_fn_1 = np.matmul(x_T, v)
# obj_fn = np.matmul(obj_fn_1, x)

In [9]:
# obj_fn.item()

In [10]:
# model.setObjective(obj_fn.item(), GRB.MINIMIZE) # minimize the variance of the portfolio return

In [11]:
obj_fn = 0.0144*x_1**2 + 0.04*x_2**2 + 0.0025*x_3**2 + 0.0288*x_1*x_2 + 0.0024*x_1*x_3 + 0.01*x_2*x_3
model.setObjective(obj_fn, GRB.MINIMIZE)

In [12]:
# add constraints
model.addConstr(x_1 + x_2 + x_3 == 1)


<gurobi.Constr *Awaiting Model Update*>

In [13]:
model.setParam("outputFlag", False)
model.optimize()

In [14]:
print(f"Optimization is done. Objective Function value: {model.ObjVal}")

Optimization is done. Objective Function value: 0.0023834518368321387


In [15]:
for v in model.getVars():
    print(f"{v.varName}:{v.x}")

x1:0.08965548800011253
x2:5.146916859888827e-07
x3:0.9103439973082008


In [17]:
model.write("qp_asset_allocation.lp")
